In [1]:
import cv2, os, face_recognition
import numpy as np
import sklearn.linear_model as sk
import cv2
from pyspark.sql.types import ArrayType, IntegerType, DoubleType, ByteType, BinaryType
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml.classification import LogisticRegressionModel
from PIL import Image
from io import BytesIO
from pyspark.sql.functions import udf


os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1  pyspark-shell'

In [2]:
SparkContext.setSystemProperty('spark.executor.memory', '4g')
SparkContext.setSystemProperty('spark.driver.memory', '2g')
spark = SparkSession \
    .builder \
    .appName("Streaming from Kafka") \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .config("spark.memory.offHeap.enabled","true") \
    .config("spark.memory.offHeap.size","8g")\
    .config("spark.sql.shuffle.partitions", 5) \
    .config("spark.network.timeout","360000ms")\
    .config("spark.executor.heartbeatInterval","300000ms")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .master("local[*]") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

your 131072x1 screen size is bogus. expect trouble
24/05/19 21:41:00 WARN Utils: Your hostname, LAPTOP-390RNSVU resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/05/19 21:41:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ntp2003/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ntp2003/.ivy2/cache
The jars for the packages stored in: /home/ntp2003/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-213a96c9-ae7d-4fd5-b041-c85be2fe5c5e;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.spark#spark-sql-kafka-0

In [3]:
streaming_df = spark.readStream\
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "test") \
    .option("startingOffsets", "earliest") \
    .option("maxOffsetsPerTrigger", 10)\
    .option("failOnDataLoss","false")\
    .load()

In [4]:
streaming_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:

lr = LogisticRegressionModel.load('lrmodel')

model = sk.LogisticRegression(C=1/lr.getRegParam())

model.coef_ = lr.coefficientMatrix.toArray()
model.intercept_= lr.interceptVector.toArray()
model.classes_ = np.asarray([0.0,1.0,2.0,3.0])
index_to_label = ['hai', 'phat', 'huy', 'bao']

In [6]:
def bin2image(bin):
    stream = BytesIO(bin)
    image = np.array(Image.open(stream).convert('RGB'))
    return np.reshape(image, (480,600,3)).tolist()

udf_bin2image = udf(bin2image, ArrayType(ArrayType(ArrayType(ByteType()))))


In [7]:


def trans_detect(frame):
    _frame = np.asarray(frame, dtype=np.uint8)
    face_locations = face_recognition.face_locations(_frame, number_of_times_to_upsample=0)
    
    
    if len(face_locations) > 0:
        face_encodings = face_recognition.face_encodings(_frame, face_locations,0)
        predicts = model.predict_proba(face_encodings)
        for i, face_location in enumerate(face_locations):
            top, right, bottom, left = face_location
            coords = [left, top, right-left, bottom - top]
            
            flg = True
            for j in range(0,len(index_to_label)):
                if predicts[i][j] > 0.7:
                    cv2.rectangle(_frame, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0, 255, 0), 2)
                    cv2.rectangle(_frame, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+30), (0, 255, 0), 2)
                    cv2.putText(_frame, f'{index_to_label[j]} ({predicts[i][j]:.2})', (coords[0]+15, coords[1]+20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                    flg = False
                    break
            if flg:
                cv2.rectangle(_frame, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (255, 0, 0), 2)
                cv2.rectangle(_frame, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+30), (255, 0, 0), 2)
                cv2.putText(_frame, 'Name: Unknown', (coords[0]+15, coords[1]+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                
    rt, buffer = cv2.imencode('.jpg', _frame)
    return buffer.tobytes()

udf_trans_detect = udf(trans_detect, BinaryType())

def foreach_batch_function(df, epoch_id):
    print(epoch_id)
    trans0_df = df.withColumn("ImageArray", udf_bin2image(df["value"])).select("ImageArray","key")
    trans1_df = trans0_df.withColumn("value", udf_trans_detect(trans0_df["ImageArray"]))
    trans2_df = trans1_df.select("key","value")
    trans2_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
    .write \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "image1") \
    .save()

In [8]:
streaming_df.writeStream.outputMode("append").trigger(availableNow = True)\
    .foreachBatch(foreach_batch_function).start().awaitTermination()

0


1


2


3


4


...

24


25


26


27


28
